In [1]:
!pip install openpyxl boto3 confluent_kafka google-cloud-bigquery pandas pyarrow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 5.1 MB/s eta 0:00:00


In [8]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery

project_id = "able-balm-454718-n8"  # ✅ Your GCP project ID
print("Your Project ID:", project_id)

client = bigquery.Client(project=project_id)


Your Project ID: able-balm-454718-n8


In [16]:
# Authenticate and import dependencies
from google.colab import auth
auth.authenticate_user()

import os
import requests
import zipfile
import pandas as pd
from urllib.parse import urlparse, unquote
from datetime import datetime
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# GCP BigQuery config
project_id = "able-balm-454718-n8"
dataset_id = "battery_sandbox"
table_prefix = "battery_data"

client = bigquery.Client(project=project_id)

# Ensure dataset exists
try:
    client.get_dataset(dataset_id)
    print(f"✅ Dataset `{dataset_id}` already exists.")
except NotFound:
    dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")
    dataset = client.create_dataset(dataset)
    print(f"✅ Created dataset `{dataset_id}`")

# Dataset URLs
dataset_urls = [
    "https://web.calce.umd.edu/batteries/data/SP1_Initial%20capacity_10_16_2015.zip",
    "https://web.calce.umd.edu/batteries/data/SP2_25C_FUDS.zip"
]

# Process and upload
for zip_url in dataset_urls:
    parsed_url = urlparse(zip_url)
    zip_filename = os.path.basename(unquote(parsed_url.path))
    zip_path = os.path.join("battery_data", zip_filename)
    extract_dir = os.path.join("battery_data", os.path.splitext(zip_filename)[0])

    os.makedirs("battery_data", exist_ok=True)

    print(f"\n📥 Downloading: {zip_filename}")
    if not os.path.exists(zip_path):
        r = requests.get(zip_url, verify=False)
        with open(zip_path, "wb") as f:
            f.write(r.content)
        print(f"✅ Downloaded to: {zip_path}")
    else:
        print("⚠️ ZIP already exists. Skipping download.")

    print(f"📂 Extracting to: {extract_dir}")
    if not os.path.exists(extract_dir):
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(extract_dir)
    else:
        print("⚠️ Directory already exists. Skipping extraction.")

    files = os.listdir(extract_dir)
    target_file = next((os.path.join(extract_dir, f) for f in files if f.lower().endswith((".xlsx", ".xls"))), None)
    if not target_file:
        raise FileNotFoundError(f"❌ No Excel file found in {extract_dir}.")

    print(f"📊 Reading Excel: {target_file}")
    df = pd.read_excel(target_file, engine="openpyxl")
    df = df.dropna(how='all').reset_index(drop=True)

    # ✅ Clean column names
    df.columns = [
        str(col).strip().lower()
        .replace(" ", "_")
        .replace("(", "")
        .replace(")", "")
        .replace("/", "_")
        .replace("%", "percent")
        for col in df.columns
    ]
    df = df.loc[:, ~df.columns.str.contains('^unnamed', case=False)]

    # ✅ Add metadata
    df["id"] = df.index
    df["source_file"] = zip_filename
    df["loaded_at"] = pd.Timestamp.utcnow()

    # ✅ Convert all object columns to string (Arrow safe)
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)

    # ✅ Convert likely ISO-like datetime strings to datetime64
    for col in df.columns:
      if df[col].dtype == "object":
          try:
              if df[col].str.match(r"^\d{4}-\d{2}-\d{2}").any():
                  df[col] = pd.to_datetime(df[col], errors="coerce")
          except Exception:
              pass

    # ✅ Final table ID
    clean_table_name = os.path.splitext(zip_filename)[0].replace('-', '_').replace(' ', '_').lower()
    table_id = f"{project_id}.{dataset_id}.{table_prefix}_{clean_table_name}"

    print(f"🚀 Uploading to BigQuery: {table_id}")
    job_config = bigquery.LoadJobConfig(
        autodetect=True,
        write_disposition="WRITE_TRUNCATE"
    )
    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()
    print(f"✅ Uploaded to BigQuery: {table_id}")


✅ Dataset `battery_sandbox` already exists.

📥 Downloading: SP1_Initial capacity_10_16_2015.zip
⚠️ ZIP already exists. Skipping download.
📂 Extracting to: battery_data/SP1_Initial capacity_10_16_2015
⚠️ Directory already exists. Skipping extraction.
📊 Reading Excel: battery_data/SP1_Initial capacity_10_16_2015/10_16_2015_Initial capacity_SP20-1.xlsx
🚀 Uploading to BigQuery: able-balm-454718-n8.battery_sandbox.battery_data_sp1_initial_capacity_10_16_2015
✅ Uploaded to BigQuery: able-balm-454718-n8.battery_sandbox.battery_data_sp1_initial_capacity_10_16_2015

📥 Downloading: SP2_25C_FUDS.zip
⚠️ ZIP already exists. Skipping download.
📂 Extracting to: battery_data/SP2_25C_FUDS
⚠️ Directory already exists. Skipping extraction.
📊 Reading Excel: battery_data/SP2_25C_FUDS/11_06_2015_SP20-2_FUDS_80SOC.xls
🚀 Uploading to BigQuery: able-balm-454718-n8.battery_sandbox.battery_data_sp2_25c_fuds


<ipython-input-16-1121c78feb66>:99: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")


✅ Uploaded to BigQuery: able-balm-454718-n8.battery_sandbox.battery_data_sp2_25c_fuds


In [ ]:
# Step 1: Install PySpark and BigQuery Connector
!pip install pyspark --quiet
!pip install -U google-cloud-bigquery --quiet

In [22]:
from pyspark.sql import SparkSession

project_id = "able-balm-454718-n8"
dataset_id = "battery_sandbox"
table_name = "battery_data_sp1_initial_capacity_10_16_2015"

spark = SparkSession.builder \
    .appName("BigQueryRead") \
    .config("spark.jars.packages", "com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.30.0") \
    .getOrCreate()

df_raw = spark.read.format("bigquery") \
    .option("project", project_id) \
    .option("parentProject", project_id) \
    .option("table", f"{dataset_id}.{table_name}") \
    .load()

df_raw.printSchema()
df_raw.show(5)



root
 |-- data_point: long (nullable = true)
 |-- test_times: double (nullable = true)
 |-- date_time: string (nullable = true)
 |-- step_times: double (nullable = true)
 |-- step_index: long (nullable = true)
 |-- cycle_index: long (nullable = true)
 |-- currenta: double (nullable = true)
 |-- voltagev: double (nullable = true)
 |-- charge_capacityah: double (nullable = true)
 |-- discharge_capacityah: double (nullable = true)
 |-- charge_energywh: double (nullable = true)
 |-- discharge_energywh: double (nullable = true)
 |-- dv_dtv_s: double (nullable = true)
 |-- internal_resistanceohm: double (nullable = true)
 |-- is_fc_data: long (nullable = true)
 |-- ac_impedanceohm: long (nullable = true)
 |-- aci_phase_angledeg: long (nullable = true)
 |-- id: long (nullable = true)
 |-- source_file: string (nullable = true)
 |-- loaded_at: timestamp (nullable = true)

+----------+------------------+-------------------+-----------------+----------+-----------+----------------+---------------

In [29]:
from pyspark.sql.functions import col, count, countDistinct, when, isnan, to_timestamp
from pyspark.sql.types import TimestampType

# --- Step 3: Display Columns and Schema ---
phys_cols = df_raw.columns
print("📋 All columns:", phys_cols)
print("📊 Raw DataFrame Row Count:", df_raw.count())
df_raw.printSchema()

# --- Step 4: Ensure 'date_time' is a proper Timestamp ---
if "date_time" in df_raw.columns:
    df_raw = df_raw.withColumn("date_time", to_timestamp(col("date_time")))

# --- Step 5: Drop rows with any NULLs across all columns ---
df_cleaned = df_raw.dropna(subset=phys_cols)
print("📉 Cleaned DataFrame Row Count:", df_cleaned.count())

# --- Step 6: Show null counts (safe for all types, no isnan for non-numeric) ---
print("🧹 Missing value counts per column:")
null_counts = df_cleaned.select([
    count(when(col(c).isNull(), c)).alias(c) for c in phys_cols
])
null_counts.show(truncate=False)

# --- Step 7: Count distinct values for constant column detection ---
print("🔍 Counting unique values per column...")
n_unique = df_cleaned.select([
    countDistinct(col(c)).alias(c) for c in df_cleaned.columns
]).collect()[0].asDict()

constant_cols = [col_name for col_name, unique_count in n_unique.items() if unique_count <= 1]
print("🧺 Dropping constant columns:", constant_cols)

df_cleaned = df_cleaned.drop(*constant_cols)

# --- Step 8: Final schema and data preview ---
print("✅ Final cleaned schema:")
df_cleaned.printSchema()
df_cleaned.show(5)


📋 All columns: ['data_point', 'test_times', 'date_time', 'step_times', 'step_index', 'cycle_index', 'currenta', 'voltagev', 'charge_capacityah', 'discharge_capacityah', 'charge_energywh', 'discharge_energywh', 'dv_dtv_s', 'internal_resistanceohm', 'is_fc_data', 'ac_impedanceohm', 'aci_phase_angledeg', 'id', 'source_file', 'loaded_at']
📊 Raw DataFrame Row Count: 27602
root
 |-- data_point: long (nullable = true)
 |-- test_times: double (nullable = true)
 |-- date_time: string (nullable = true)
 |-- step_times: double (nullable = true)
 |-- step_index: long (nullable = true)
 |-- cycle_index: long (nullable = true)
 |-- currenta: double (nullable = true)
 |-- voltagev: double (nullable = true)
 |-- charge_capacityah: double (nullable = true)
 |-- discharge_capacityah: double (nullable = true)
 |-- charge_energywh: double (nullable = true)
 |-- discharge_energywh: double (nullable = true)
 |-- dv_dtv_s: double (nullable = true)
 |-- internal_resistanceohm: double (nullable = true)
 |-- is

In [27]:
print("Raw DataFrame Row Count:", df_raw.count())
print("All columns:", df_raw.columns)
df_raw.printSchema()

print("Cleaned DataFrame Row Count:", df_cleaned.count())
print("All columns:", df_cleaned.columns)
df_cleaned.printSchema()


Raw DataFrame Row Count: 27602
All columns: ['data_point', 'test_times', 'date_time', 'step_times', 'step_index', 'cycle_index', 'currenta', 'voltagev', 'charge_capacityah', 'discharge_capacityah', 'charge_energywh', 'discharge_energywh', 'dv_dtv_s', 'internal_resistanceohm', 'is_fc_data', 'ac_impedanceohm', 'aci_phase_angledeg', 'id', 'source_file', 'loaded_at']
root
 |-- data_point: long (nullable = true)
 |-- test_times: double (nullable = true)
 |-- date_time: string (nullable = true)
 |-- step_times: double (nullable = true)
 |-- step_index: long (nullable = true)
 |-- cycle_index: long (nullable = true)
 |-- currenta: double (nullable = true)
 |-- voltagev: double (nullable = true)
 |-- charge_capacityah: double (nullable = true)
 |-- discharge_capacityah: double (nullable = true)
 |-- charge_energywh: double (nullable = true)
 |-- discharge_energywh: double (nullable = true)
 |-- dv_dtv_s: double (nullable = true)
 |-- internal_resistanceohm: double (nullable = true)
 |-- is_fc_

In [30]:
df_cleaned.write \
    .format("bigquery") \
    .option("writeMethod", "direct") \
    .option("writeDisposition", "WRITE_TRUNCATE") \
    .option("project", project_id) \
    .option("parentProject", project_id) \
    .option("table", f"{dataset_id}.battery_data_sp1_cleaned") \
    .mode("overwrite") \
    .save()
